In [24]:
import torch
import numpy as np
import re
from transformers import AutoTokenizer, AutoModel, AutoConfig
from nltk import word_tokenize
import re
import tensorflow as tf
import nltk.corpus 
import nltk
from keras.utils import np_utils
from numpy import savez_compressed

Using TensorFlow backend.


Retrieving dataset (George Orweil's 1984)

In [14]:
nltk.download('mte_teip5')
data = nltk.corpus.multext_east.tagged_sents("oana-en.xml", "universal")

[nltk_data] Downloading package mte_teip5 to
[nltk_data]     /Users/Mahwish/nltk_data...
[nltk_data]   Package mte_teip5 is already up-to-date!


Retrieving saved variables from below queries

In [51]:
from numpy import load
# load dict of arrays + extract the first array
dict_data = load('BERT_labels_xx.npz')
xx = dict_data['arr_0'] # contextual embeddings tensor saved for range(0,400)
dict_data = load('BERT_labels_yy.npz')
yy = dict_data['arr_0'] # corresponding labels saved in list
dict_data = load('BERT_bubu2_yy.npz')
bubu2 = dict_data['arr_0'] # to check the text corresponding to label
dict_data = load('BERT_extra_set_list_xx.npz')
extra_set_list = dict_data['arr_0'] # to check the text corresponding to embedding
dict_data = load('BERT_dev_labels_yy.npz')
dev_labels = dict_data['arr_0'] # previously computed labels (extension of yy)

Checking transformer version is correct -- older version causes problems

In [9]:
#!pip install transformers --upgrade
#import transformers
#print(transformers.__version__)
#4.9.0.dev0 + required

Set seed

In [4]:
#for reproducibility
seed=20
cust_seed = np.random.seed(seed)

Checking how indexing works when splitting sentences

In [26]:
#Testing splitting methods
st = 'The cat sat on the mat'
st.split(" ").index('mat')

5

The below function creates chunks of words splitting the text after 40 tokens. Chose this based on average sentence length to ensure 'context' is kept to give an idea of syntactic structure of the sentence when retrieving a particular words embedding, but not too much context as to diminish the attention given to the particular tokens. This should help when POS tagging (a noun came before i'th token, punctuation after, etc).

In [15]:
def set_creator(chunk):
    text = []
    tag_counter = 0
    for idx3, k in enumerate(data):
        for idx4, m in enumerate(k):
            if chunk*40 <= tag_counter < (chunk+1)*40:
                text.append(m[0])
            tag_counter += 1
    text = ' '.join(text)
    input_string = text
    
    #Here I'm trying to experiment on how to pad the string so that the punctuation is separated when grabbing the embedding
    # if the string isn't padded, it'll try to find 'pumpkin.' instead of 'pumpkin', '.'
    pattern = '([:;.,!?()])'
    input_string = re.sub(pattern, r' \1 ', input_string)
    input_string = re.sub('\s\s*',' ',input_string)
    input_string = re.sub(" '","'",input_string)
    input_string = re.sub("s' ","s ' ",input_string)
    return input_string   
#print(set_creator())

The two functions below (from huggingface blog) used to retrieve the contextual embeddings of each token given a context string generated from the above. Comments added for info.

In [45]:
def get_hidden_states(encoded, token_ids_word, model, layers):
    """Push input IDs through model. Stack and sum 'layers' (last four by default).
       Select only those subword token outputs that belong to our word of interest
       and average them."""
    with torch.no_grad():
        output = model(**encoded)
        #print(output)
        
    # Gets all hidden states from model
    states = output.hidden_states
    # Stack and sum all requested layers (defined which ones later - using -4, -3, -2, -1)
    output = torch.stack([states[o] for o in layers]).sum(0).squeeze()
    
    print('output',np.shape(output))
    # Only select the tokens that constitute the requested word (since subtokens can be generated)
    word_tokens_output = output[token_ids_word]
    #print('word_tokens_output',word_tokens_output[0][0], np.shape(word_tokens_output))
    #print('avg subcomp word',word_tokens_output.mean(dim=0)[0],np.shape((word_tokens_output.mean(dim=0))))
    return word_tokens_output.mean(dim=0) # this averages the sub components of the word

def get_word_vector(sent, idx, tokenizer, model, layers):
    """Get a word vector by first tokenizing the input sentence, getting all token idxs
       that make up the word of interest, and then 'get_hidden_states'."""
    encoded = tokenizer.encode_plus(sent, return_tensors="pt")
    print('encoded',encoded)
    # get all token idxs that belong to the word of interest
    token_ids_word = np.where(np.array(encoded.word_ids()) == idx)
    #print('token_ids_word',token_ids_word)
    print('encoded.word_ids-idx',(np.array(encoded.word_ids()) == idx))
    return get_hidden_states(encoded, token_ids_word, model, layers)

Example below printing sections of above code, and retrieving the contextual embeddings of word 'mat' from sentence 'The cat sat on the mat'.

In [13]:
layers = [-4, -3, -2, -1]
tokenizer = AutoTokenizer.from_pretrained("vblagoje/bert-english-uncased-finetuned-pos")
config = AutoConfig.from_pretrained("vblagoje/bert-english-uncased-finetuned-pos", output_hidden_states=True)
model = AutoModel.from_pretrained("vblagoje/bert-english-uncased-finetuned-pos", config=config)

sent = "The cat sat on the mat" 
idx = sent.split(" ").index('mat')
word_embedding = get_word_vector(sent, idx, tokenizer, model, layers)

Some weights of the model checkpoint at vblagoje/bert-english-uncased-finetuned-pos were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


encoded {'input_ids': tensor([[  101,  1996,  4937,  2938,  2006,  1996, 13523,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}
encoded.word_ids-idx [False False False False False False  True False]
output torch.Size([8, 768])
word_tokens_output tensor(2.5789) torch.Size([1, 768])
avg subcomp word tensor(2.5789) torch.Size([768])


Below script retrieves embeddings for each token within a context chunk ( of length 40), and concatenates them into a contextual embedding tensor (tens). The tens for which results are beneath is for range(0,400) but this takes a while to run so I've changed to range (0,10) for demonstration purpose.

In [48]:
# Use last four layers by default
counter=0
layers = [-4, -3, -2, -1]
tokenizer = AutoTokenizer.from_pretrained("vblagoje/bert-english-uncased-finetuned-pos")
config = AutoConfig.from_pretrained("vblagoje/bert-english-uncased-finetuned-pos", output_hidden_states=True)
model = AutoModel.from_pretrained("vblagoje/bert-english-uncased-finetuned-pos", output_hidden_states=True)
tens = torch.empty((0), dtype=torch.int32)
extra_set_list = []
for q in range(0,10):
    input_string = set_creator(q)
    ww = input_string.split(" ")
    print('q',q)

    for ix, i in enumerate(ww):
        print(ix,i)
        #print(list_labels[ix])
        if i != '':
            extra_set_list.append(i)
            idx = ix
            word_embedding = get_word_vector(input_string,idx,tokenizer,model,layers)
            
            if (ix == 0 & q == 0): # CHANGE THIS 
                tens = word_embedding[None,:]
                print(word_embedding)
            else:
                tens = torch.cat((tens, word_embedding[None,:]),0)
                print(ix,i, 'tens shape',np.shape(tens))
                print(word_embedding[0])
                print('counter',counter)
            
            counter+=1

Some weights of the model checkpoint at vblagoje/bert-english-uncased-finetuned-pos were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


q 0
0 It
encoded {'input_ids': tensor([[  101,  2009,  2001,  1037,  4408,  3147,  2154,  1999,  2258,  1010,
          1998,  1996, 20940,  2020,  8478,  7093,  1012, 10180,  3044,  1010,
          2010,  5413, 16371, 17269,  2046,  2010,  7388,  1999,  2019,  3947,
          2000,  4019,  1996, 25047,  3612,  1010,  5707,  2855,  2083,  1996,
          3221,  4303,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
encoded.word_ids-idx [False  True False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False]


output torch.Size([43, 768])
1 was tens shape torch.Size([2, 768])
tensor(0.5775)
counter 1
2 a
encoded {'input_ids': tensor([[  101,  2009,  2001,  1037,  4408,  3147,  2154,  1999,  2258,  1010,
          1998,  1996, 20940,  2020,  8478,  7093,  1012, 10180,  3044,  1010,
          2010,  5413, 16371, 17269,  2046,  2010,  7388,  1999,  2019,  3947,
          2000,  4019,  1996, 25047,  3612,  1010,  5707,  2855,  2083,  1996,
          3221,  4303,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
encoded.word_ids-idx [False False False  True False False False False False False False False
 False False False False False False False False False False False False
 False False False False False

output torch.Size([43, 768])
9 and tens shape torch.Size([10, 768])
tensor(-1.8758)
counter 9
10 the
encoded {'input_ids': tensor([[  101,  2009,  2001,  1037,  4408,  3147,  2154,  1999,  2258,  1010,
          1998,  1996, 20940,  2020,  8478,  7093,  1012, 10180,  3044,  1010,
          2010,  5413, 16371, 17269,  2046,  2010,  7388,  1999,  2019,  3947,
          2000,  4019,  1996, 25047,  3612,  1010,  5707,  2855,  2083,  1996,
          3221,  4303,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
encoded.word_ids-idx [False False False False False False False False False False False  True
 False False False False False False False False False False False False
 False False False False 

output torch.Size([43, 768])
17 Smith tens shape torch.Size([18, 768])
tensor(3.2208)
counter 17
18 ,
encoded {'input_ids': tensor([[  101,  2009,  2001,  1037,  4408,  3147,  2154,  1999,  2258,  1010,
          1998,  1996, 20940,  2020,  8478,  7093,  1012, 10180,  3044,  1010,
          2010,  5413, 16371, 17269,  2046,  2010,  7388,  1999,  2019,  3947,
          2000,  4019,  1996, 25047,  3612,  1010,  5707,  2855,  2083,  1996,
          3221,  4303,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
encoded.word_ids-idx [False False False False False False False False False False False False
 False False False False False False False  True False False False False
 False False False False

...

output torch.Size([49, 768])
8 merely tens shape torch.Size([9, 768])
tensor(0.6038)
counter 368
9 emphasized
encoded {'input_ids': tensor([[  101, 25737,  3275,  1010,  1996,  2033,  8490,  7389,  7971,  1997,
          2010,  2303,  6414, 13155,  2011,  1996,  2630,  3452,  2015,  2029,
          2020,  1996,  6375,  1997,  1996,  2283,  1012,  2010,  2606,  2001,
          2200,  4189,  1010,  2010,  2227,  8100,  6369, 20023,  2063,  1010,
          2010,  3096,  5931,  6675,  2011, 20392,  7815,  1998,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1]])}
encoded.word_ids-idx [False False False False False False False False False False False

output torch.Size([49, 768])
15 were tens shape torch.Size([16, 768])
tensor(2.5366)
counter 375
16 the
encoded {'input_ids': tensor([[  101, 25737,  3275,  1010,  1996,  2033,  8490,  7389,  7971,  1997,
          2010,  2303,  6414, 13155,  2011,  1996,  2630,  3452,  2015,  2029,
          2020,  1996,  6375,  1997,  1996,  2283,  1012,  2010,  2606,  2001,
          2200,  4189,  1010,  2010,  2227,  8100,  6369, 20023,  2063,  1010,
          2010,  3096,  5931,  6675,  2011, 20392,  7815,  1998,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1]])}
encoded.word_ids-idx [False False False False False False False False False False False False

output torch.Size([49, 768])
22 His tens shape torch.Size([23, 768])
tensor(3.8306)
counter 382
23 hair
encoded {'input_ids': tensor([[  101, 25737,  3275,  1010,  1996,  2033,  8490,  7389,  7971,  1997,
          2010,  2303,  6414, 13155,  2011,  1996,  2630,  3452,  2015,  2029,
          2020,  1996,  6375,  1997,  1996,  2283,  1012,  2010,  2606,  2001,
          2200,  4189,  1010,  2010,  2227,  8100,  6369, 20023,  2063,  1010,
          2010,  3096,  5931,  6675,  2011, 20392,  7815,  1998,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1]])}
encoded.word_ids-idx [False False False False False False False False False False False False

output torch.Size([49, 768])
29 face tens shape torch.Size([30, 768])
tensor(4.8590)
counter 389
30 naturally
encoded {'input_ids': tensor([[  101, 25737,  3275,  1010,  1996,  2033,  8490,  7389,  7971,  1997,
          2010,  2303,  6414, 13155,  2011,  1996,  2630,  3452,  2015,  2029,
          2020,  1996,  6375,  1997,  1996,  2283,  1012,  2010,  2606,  2001,
          2200,  4189,  1010,  2010,  2227,  8100,  6369, 20023,  2063,  1010,
          2010,  3096,  5931,  6675,  2011, 20392,  7815,  1998,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1]])}
encoded.word_ids-idx [False False False False False False False False False False False

output torch.Size([49, 768])
36 by tens shape torch.Size([37, 768])
tensor(-2.0391)
counter 396
37 coarse
encoded {'input_ids': tensor([[  101, 25737,  3275,  1010,  1996,  2033,  8490,  7389,  7971,  1997,
          2010,  2303,  6414, 13155,  2011,  1996,  2630,  3452,  2015,  2029,
          2020,  1996,  6375,  1997,  1996,  2283,  1012,  2010,  2606,  2001,
          2200,  4189,  1010,  2010,  2227,  8100,  6369, 20023,  2063,  1010,
          2010,  3096,  5931,  6675,  2011, 20392,  7815,  1998,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1]])}
encoded.word_ids-idx [False False False False False False False False False False False Fal

This part deals with POS tags from NLTK corpus (data variable instantiated above)

### Label creators


The below creates the labels corresponding to contextual embeddings for tokens tensor above. Some parts are manual, as some parts are split differently above, as they are in the NLTK corpus. For example, '3,000' versus '3'  ','  '000'  splits differently for both. Same for '...' versus '.' ,'.', '.' .

In [1565]:
## ----------- II ----------- #
#test label creator
bubu2=[]
dev_labels = []
tag_counter = 0
counter_label = 0
for idx3, k in enumerate(data):
    #print(idx3, k)
    for idx4, m in enumerate(k):
        #print(tag_counter,m)
        if (40*0)-1 < tag_counter < 40*401:
            if tag_counter in(5920,7200):
                print(tag_counter,m)
                bubu2.append("'s")
                dev_labels.append('VERB')
            elif tag_counter == 10760:
                bubu2.append("'s")
                dev_labels.append('ADP')
                print("s',ADP") 
            elif tag_counter == 8360:
                bubu2.append("'m")
                dev_labels.append('VERB')
                print("m,VERB")
            elif tag_counter == 11286:
                bubu2.append("3")
                dev_labels.append('NUM')
                print("3,NUM")
                bubu2.append(",")
                dev_labels.append('')
                bubu2.append("000")
                dev_labels.append('NUM')
            elif tag_counter == 13297:
                bubu2.append("W")
                dev_labels.append('NOUN')
                bubu2.append(".")
                dev_labels.append('')
            elif m[0] !=  "'s" and  m[0] != "'ve" and  m[0] != "'t" and m[0] !=  "'d" and m[0] !=  "'m" and m[0] !=  "'re" and m[0] !=  "'ave"and m[0] !=  "'ll" and m[0] !=  "'em" :
                if m[0] == '...':
                    bubu2.append('.')
                    dev_labels.append('')
                    bubu2.append('.')
                    dev_labels.append('')
                    print('')  
                elif tag_counter in (13888,13895,13907,13913,16062):
                    bubu2.append('17')
                    dev_labels.append('NUM')
                    bubu2.append('.')
                    dev_labels.append('')
                    bubu2.append('3')
                    dev_labels.append('NUM')
                    bubu2.append('.')
                    dev_labels.append('')
                bubu2.append(m[0])
                dev_labels.append(m[1])
                #print(ww[tag_counter])
            print(tag_counter,m)
        tag_counter += 1
        counter_label +=1

0 ('It', 'PRON')
1 ('was', 'VERB')
2 ('a', 'DET')
3 ('bright', 'ADJ')
4 ('cold', 'ADJ')
5 ('day', 'NOUN')
6 ('in', 'ADP')
7 ('April', 'NOUN')
8 (',', '')
9 ('and', 'CONJ')
10 ('the', 'DET')
11 ('clocks', 'NOUN')
12 ('were', 'VERB')
13 ('striking', 'VERB')
14 ('thirteen', 'NUM')
15 ('.', '')
16 ('Winston', 'NOUN')
17 ('Smith', 'NOUN')
18 (',', '')
19 ('his', 'DET')
20 ('chin', 'NOUN')
21 ('nuzzled', 'VERB')
22 ('into', 'ADP')
23 ('his', 'DET')
24 ('breast', 'NOUN')
25 ('in', 'ADP')
26 ('an', 'DET')
27 ('effort', 'NOUN')
28 ('to', 'ADP')
29 ('escape', 'VERB')
30 ('the', 'DET')
31 ('vile', 'ADJ')
32 ('wind', 'NOUN')
33 (',', '')
34 ('slipped', 'VERB')
35 ('quickly', 'ADV')
36 ('through', 'ADP')
37 ('the', 'DET')
38 ('glass', 'NOUN')
39 ('doors', 'NOUN')
40 ('of', 'ADP')
41 ('Victory', 'NOUN')
42 ('Mansions', 'NOUN')
43 (',', '')
44 ('though', 'CONJ')
45 ('not', 'ADV')
46 ('quickly', 'ADV')
47 ('enough', 'ADV')
48 ('to', 'ADP')
49 ('prevent', 'VERB')
50 ('a', 'DET')
51 ('swirl', 'NOUN')
52

893 ('of', 'ADP')
894 ('timber', 'NOUN')
895 (',', '')
896 ('their', 'DET')
897 ('windows', 'NOUN')
898 ('patched', 'VERB')
899 ('with', 'ADP')
900 ('cardboard', 'NOUN')
901 ('and', 'CONJ')
902 ('their', 'DET')
903 ('roofs', 'NOUN')
904 ('with', 'ADP')
905 ('corrugated', 'ADJ')
906 ('iron', 'NOUN')
907 (',', '')
908 ('their', 'DET')
909 ('crazy', 'ADJ')
910 ('garden', 'NOUN')
911 ('walls', 'NOUN')
912 ('sagging', 'VERB')
913 ('in', 'ADP')
914 ('all', 'DET')
915 ('directions', 'NOUN')
916 ('?', '')
917 ('And', 'CONJ')
918 ('the', 'DET')
919 ('bombed', 'ADJ')
920 ('sites', 'NOUN')
921 ('where', 'ADV')
922 ('the', 'DET')
923 ('plaster', 'NOUN')
924 ('dust', 'NOUN')
925 ('swirled', 'VERB')
926 ('in', 'ADP')
927 ('the', 'DET')
928 ('air', 'NOUN')
929 ('and', 'CONJ')
930 ('the', 'DET')
931 ('willow-herb', 'NOUN')
932 ('straggled', 'VERB')
933 ('over', 'ADP')
934 ('the', 'DET')
935 ('heaps', 'NOUN')
936 ('of', 'ADP')
937 ('rubble', 'NOUN')
938 (';', '')
939 ('and', 'CONJ')
940 ('the', 'DET')


1796 ('Its', 'DET')
1797 ('smooth', 'ADJ')
1798 ('creamy', 'ADJ')
1799 ('paper', 'NOUN')
1800 (',', '')
1801 ('a', 'DET')
1802 ('little', 'ADV')
1803 ('yellowed', 'VERB')
1804 ('by', 'ADP')
1805 ('age', 'NOUN')
1806 (',', '')
1807 ('was', 'VERB')
1808 ('of', 'ADP')
1809 ('a', 'DET')
1810 ('kind', 'NOUN')
1811 ('that', 'PRON')
1812 ('had', 'VERB')
1813 ('not', 'ADV')
1814 ('been', 'VERB')
1815 ('manufactured', 'VERB')
1816 ('for', 'ADP')
1817 ('at', 'ADP')
1818 ('least', 'ADJ')
1819 ('forty', 'NUM')
1820 ('years', 'NOUN')
1821 ('past', 'ADV')
1822 ('.', '')
1823 ('He', 'PRON')
1824 ('could', 'VERB')
1825 ('guess', 'VERB')
1826 (',', '')
1827 ('however', 'ADV')
1828 (',', '')
1829 ('that', 'CONJ')
1830 ('the', 'DET')
1831 ('book', 'NOUN')
1832 ('was', 'VERB')
1833 ('much', 'ADV')
1834 ('older', 'ADJ')
1835 ('than', 'ADP')
1836 ('that', 'PRON')
1837 ('.', '')
1838 ('He', 'PRON')
1839 ('had', 'VERB')
1840 ('seen', 'VERB')
1841 ('it', 'PRON')
1842 ('lying', 'VERB')
1843 ('in', 'ADP')
1844 (

2553 ('The', 'DET')
2554 ('seconds', 'NOUN')
2555 ('were', 'VERB')
2556 ('ticking', 'VERB')
2557 ('by', 'ADV')
2558 ('.', '')
2559 ('He', 'PRON')
2560 ('was', 'VERB')
2561 ('conscious', 'ADJ')
2562 ('of', 'ADP')
2563 ('nothing', 'PRON')
2564 ('except', 'ADP')
2565 ('the', 'DET')
2566 ('blankness', 'NOUN')
2567 ('of', 'ADP')
2568 ('the', 'DET')
2569 ('page', 'NOUN')
2570 ('in', 'ADP')
2571 ('front', 'NOUN')
2572 ('of', 'ADP')
2573 ('him', 'PRON')
2574 (',', '')
2575 ('the', 'DET')
2576 ('itching', 'NOUN')
2577 ('of', 'ADP')
2578 ('the', 'DET')
2579 ('skin', 'NOUN')
2580 ('above', 'ADP')
2581 ('his', 'DET')
2582 ('ankle', 'NOUN')
2583 (',', '')
2584 ('the', 'DET')
2585 ('blaring', 'NOUN')
2586 ('of', 'ADP')
2587 ('the', 'DET')
2588 ('music', 'NOUN')
2589 (',', '')
2590 ('and', 'CONJ')
2591 ('a', 'DET')
2592 ('slight', 'ADJ')
2593 ('booziness', 'NOUN')
2594 ('caused', 'VERB')
2595 ('by', 'ADP')
2596 ('the', 'DET')
2597 ('gin', 'NOUN')
2598 ('.', '')
2599 ('Suddenly', 'ADV')
2600 ('he', 'P

3662 ('was', 'VERB')
3663 ('intrigued', 'VERB')
3664 ('by', 'ADP')
3665 ('the', 'DET')
3666 ('contrast', 'NOUN')
3667 ('between', 'ADP')
3668 ("O'Brien", 'NOUN')
3669 ("'s", 'ADP')
3670 ('urbane', 'ADJ')
3671 ('manner', 'NOUN')
3672 ('and', 'CONJ')
3673 ('his', 'DET')
3674 ('prize-fighter', 'NOUN')
3675 ("'s", 'ADP')
3676 ('physique', 'NOUN')
3677 ('.', '')
3678 ('Much', 'ADV')
3679 ('more', 'ADV')
3680 ('it', 'PRON')
3681 ('was', 'VERB')
3682 ('because', 'CONJ')
3683 ('of', 'ADP')
3684 ('a', 'DET')
3685 ('secretly', 'ADV')
3686 ('held', 'ADJ')
3687 ('belief', 'NOUN')
3688 ('-', '')
3689 ('or', 'CONJ')
3690 ('perhaps', 'ADV')
3691 ('not', 'ADV')
3692 ('even', 'ADV')
3693 ('a', 'DET')
3694 ('belief', 'NOUN')
3695 (',', '')
3696 ('merely', 'ADV')
3697 ('a', 'DET')
3698 ('hope', 'NOUN')
3699 ('-', '')
3700 ('that', 'CONJ')
3701 ("O'Brien", 'NOUN')
3702 ("'s", 'ADP')
3703 ('political', 'ADJ')
3704 ('orthodoxy', 'NOUN')
3705 ('was', 'VERB')
3706 ('not', 'ADV')
3707 ('perfect', 'ADJ')
3708 (

...

12049 ('his', 'DET')
12050 ('eyes', 'NOUN')
12051 ('were', 'VERB')
12052 ('pure', 'ADJ')
12053 ('gin', 'NOUN')
12054 ('.', '')
12055 ('But', 'CONJ')
12056 ('though', 'CONJ')
12057 ('slightly', 'ADV')
12058 ('drunk', 'ADJ')
12059 ('he', 'PRON')
12060 ('was', 'VERB')
12061 ('also', 'ADV')
12062 ('suffering', 'VERB')
12063 ('under', 'ADP')
12064 ('some', 'DET')
12065 ('grief', 'NOUN')
12066 ('that', 'PRON')
12067 ('was', 'VERB')
12068 ('genuine', 'ADJ')
12069 ('and', 'CONJ')
12070 ('unbearable', 'ADJ')
12071 ('.', '')
12072 ('In', 'ADP')
12073 ('his', 'DET')
12074 ('childish', 'ADJ')
12075 ('way', 'NOUN')
12076 ('Winston', 'NOUN')
12077 ('grasped', 'VERB')
12078 ('that', 'CONJ')
12079 ('some', 'DET')
12080 ('terrible', 'ADJ')
12081 ('thing', 'NOUN')
12082 (',', '')
12083 ('something', 'PRON')
12084 ('that', 'PRON')
12085 ('was', 'VERB')
12086 ('beyond', 'ADP')
12087 ('forgiveness', 'NOUN')
12088 ('and', 'CONJ')
12089 ('could', 'VERB')
12090 ('never', 'ADV')
12091 ('be', 'VERB')
12092 ('re

12937 ('Winston', 'NOUN')
12938 ('loathed', 'VERB')
12939 ('this', 'DET')
12940 ('exercise', 'NOUN')
12941 (',', '')
12942 ('which', 'PRON')
12943 ('sent', 'VERB')
12944 ('shooting', 'ADJ')
12945 ('pains', 'NOUN')
12946 ('all', 'DET')
12947 ('the', 'DET')
12948 ('way', 'NOUN')
12949 ('from', 'ADP')
12950 ('his', 'DET')
12951 ('heels', 'NOUN')
12952 ('to', 'ADP')
12953 ('his', 'DET')
12954 ('buttocks', 'NOUN')
12955 ('and', 'CONJ')
12956 ('often', 'ADV')
12957 ('ended', 'VERB')
12958 ('by', 'ADP')
12959 ('bringing', 'VERB')
12960 ('on', 'ADP')
12961 ('another', 'DET')
12962 ('coughing', 'VERB')
12963 ('fit', 'NOUN')
12964 ('.', '')
12965 ('The', 'DET')
12966 ('half-pleasant', 'ADJ')
12967 ('quality', 'NOUN')
12968 ('went', 'VERB')
12969 ('out', 'ADP')
12970 ('of', 'ADP')
12971 ('his', 'DET')
12972 ('meditations', 'NOUN')
12973 ('.', '')
12974 ('The', 'DET')
12975 ('past', 'NOUN')
12976 (',', '')
12977 ('he', 'PRON')
12978 ('reflected', 'VERB')
12979 (',', '')
12980 ('had', 'VERB')
12981

14091 ('As', 'CONJ')
14092 ('it', 'PRON')
14093 ('happened', 'VERB')
14094 (',', '')
14095 ('the', 'DET')
14096 ('Eurasian', 'ADJ')
14097 ('Higher', 'ADJ')
14098 ('Command', 'NOUN')
14099 ('had', 'VERB')
14100 ('launched', 'VERB')
14101 ('its', 'DET')
14102 ('offensive', 'NOUN')
14103 ('in', 'ADP')
14104 ('South', 'ADJ')
14105 ('India', 'NOUN')
14106 ('and', 'CONJ')
14107 ('left', 'VERB')
14108 ('North', 'ADJ')
14109 ('Africa', 'NOUN')
14110 ('alone', 'ADJ')
14111 ('.', '')
14112 ('It', 'PRON')
14113 ('was', 'VERB')
14114 ('therefore', 'ADV')
14115 ('necessary', 'ADJ')
14116 ('to', 'ADP')
14117 ('rewrite', 'VERB')
14118 ('a', 'DET')
14119 ('paragraph', 'NOUN')
14120 ('of', 'ADP')
14121 ('Big', 'ADJ')
14122 ('Brother', 'NOUN')
14123 ("'s", 'ADP')
14124 ('speech', 'NOUN')
14125 (',', '')
14126 ('in', 'ADP')
14127 ('such', 'ADJ')
14128 ('a', 'DET')
14129 ('way', 'NOUN')
14130 ('as', 'CONJ')
14131 ('to', 'ADP')
14132 ('make', 'VERB')
14133 ('him', 'PRON')
14134 ('predict', 'VERB')
14135 ('

14778 ('instructions', 'NOUN')
14779 ('which', 'PRON')
14780 ('Winston', 'NOUN')
14781 ('received', 'VERB')
14782 (',', '')
14783 ('and', 'CONJ')
14784 ('which', 'PRON')
14785 ('he', 'PRON')
14786 ('invariably', 'ADV')
14787 ('got', 'VERB')
14788 ('rid', 'VERB')
14789 ('of', 'ADP')
14790 ('as', 'ADV')
14791 ('soon', 'ADV')
14792 ('as', 'CONJ')
14793 ('he', 'PRON')
14794 ('had', 'VERB')
14795 ('dealt', 'VERB')
14796 ('with', 'ADP')
14797 ('them', 'PRON')
14798 (',', '')
14799 ('never', 'ADV')
14800 ('stated', 'VERB')
14801 ('or', 'CONJ')
14802 ('implied', 'VERB')
14803 ('that', 'CONJ')
14804 ('an', 'DET')
14805 ('act', 'NOUN')
14806 ('of', 'ADP')
14807 ('forgery', 'NOUN')
14808 ('was', 'VERB')
14809 ('to', 'ADP')
14810 ('be', 'VERB')
14811 ('committed', 'VERB')
14812 (':', '')
14813 ('always', 'ADV')
14814 ('the', 'DET')
14815 ('reference', 'NOUN')
14816 ('was', 'VERB')
14817 ('to', 'ADP')
14818 ('slips', 'NOUN')
14819 (',', '')
14820 ('errors', 'NOUN')
14821 (',', '')
14822 ('misprints

15730 ('the', 'DET')
15731 ('whole', 'ADJ')
15732 ('operation', 'NOUN')
15733 ('at', 'ADP')
15734 ('a', 'DET')
15735 ('lower', 'ADJ')
15736 ('level', 'NOUN')
15737 ('for', 'ADP')
15738 ('the', 'DET')
15739 ('benefit', 'NOUN')
15740 ('of', 'ADP')
15741 ('the', 'DET')
15742 ('proletariat', 'NOUN')
15743 ('.', '')
15744 ('There', 'PRON')
15745 ('was', 'VERB')
15746 ('a', 'DET')
15747 ('whole', 'ADJ')
15748 ('chain', 'NOUN')
15749 ('of', 'ADP')
15750 ('separate', 'ADJ')
15751 ('departments', 'NOUN')
15752 ('dealing', 'VERB')
15753 ('with', 'ADP')
15754 ('proletarian', 'ADJ')
15755 ('literature', 'NOUN')
15756 (',', '')
15757 ('music', 'NOUN')
15758 (',', '')
15759 ('drama', 'NOUN')
15760 (',', '')
15761 ('and', 'CONJ')
15762 ('entertainment', 'NOUN')
15763 ('generally', 'ADV')
15764 ('.', '')
15765 ('Here', 'ADV')
15766 ('were', 'VERB')
15767 ('produced', 'VERB')
15768 ('rubbishy', 'ADJ')
15769 ('newspapers', 'NOUN')
15770 ('containing', 'VERB')
15771 ('almost', 'ADV')
15772 ('nothing', 'P

The below code checks whether the two match

In [50]:
for idx, x in enumerate(extra_set_list):
    if not x == bubu2[idx]:
        print(x, bubu2[idx], idx)
       #break
print(extra_set_list[15650:15660])
print(dev_labels[15650:15660])
print(tens[15650][0])
print(tens[15659][0])
print(dev_labels[15659])
print(extra_set_list[15659])

#Taking a section of the tensor, and corresponding labels that we know match for trial
xx = np.copy(tens[0:15659])
yy = np.copy(dev_labels[0:15659])

Winston's Winston 510
people's people 576
Winston's Winston 594
tomorrow's tomorrow 1392
child's child 2883
O'Brien's O'Brien 3663
prize-fighter's prize-fighter 3668
O'Brien's O'Brien 3694
one's one 3894
one's one 3906
Party's Party 4058
Winston's Winston 4128
Goldstein's Goldstein 4403
Goldstein's Goldstein 4468
O'Brien's O'Brien 4867
Goldstein's Goldstein 4934
one's one 5055
Winston's Winston 5098
one's one 5276
one's one 5299
sheep's sheep 5470
everyone's everyone 5691
. ... 5779
. ... 5784
. ... 5933
O'Brien's O'Brien 6027
O'Brien's O'Brien 6181
else's else 6188
it's it 7375
Winston's Winston 7394
It's It 7572
he'd he 7676
He's He 7693
Winston's Winston 7706
I'm I 7947
boy's boy 8119
You're You 8130
You're You 8138
You're You 8142
I'll I 8147
I'll I 8151
I'll I 8155
boy's boy 8216
They're They 8307
that's that 8318
woman's woman 8522
O'Brien's O'Brien 9011
morning's morning 9046
,'t , 9303
,'t , 9327
father's father 10355
one's one 10681
mother's mother 10783
women's women 11021
. 

IndexError: index 15650 is out of bounds for dimension 0 with size 40

Removing NaN values

In [1576]:
#Before removing any nan values (shape)
print(np.shape(xx))
print(np.shape(yy))
#After removing nan values (shape)
print(np.shape(xx[~np.isnan(xx).any(axis=1),:]))
print(np.shape(yy[~np.isnan(xx[:,0])]))

#Removing any NaN values for both x and corresponding labels
yy = yy[~np.isnan(xx[:,0])]
xx = xx[~np.isnan(xx).any(axis=1),:]

#Check shape again and that they match
print(np.shape(xx))
print(np.shape(yy))

(15922, 768)
(15922,)
(15952, 768)
(15952,)


In [1598]:
#Assigning to another variable for splitting
fun_train = xx
fun_label =  yy
np.shape(fun_train),np.shape(fun_label)

40
(15922, 768)
(15992,)


((15922, 768), (15922,))

Splitting dataset to train and test - shuffle: True

In [1581]:
from sklearn.model_selection import train_test_split
TEST_SIZE = 0.1
X_train, X_test, Y_train, Y_test = train_test_split(fun_train, fun_label, test_size=TEST_SIZE, shuffle=True,random_state=2)

Label encoder: (only required for keras sequential layers below)

In [20]:
from sklearn.preprocessing import LabelEncoder
def label_encoder(train_labels, test_labels):
	le = LabelEncoder()
	le.fit(np.concatenate((train_labels,test_labels)))
    
    #transform
	#train_label_enc = le.transform(train_labels)
	#test_label_enc = le.transform(test_labels)

    #transform + one-hot
	train_label_enc = np_utils.to_categorical(le.transform(train_labels))
	test_label_enc = np_utils.to_categorical(le.transform(test_labels))
    
	return train_label_enc, test_label_enc

In [1582]:
Y_traini, Y_testi = label_encoder(Y_train,Y_test)

Scaling to experiment below:

In [1583]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
scaler = preprocessing.StandardScaler().fit(X_train)
X_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

Below using logistic regression on training set composed of embeddings and coresponding POS tags. Encoding not required for logistic regression (inbuilt). Commented out some accuracies below for reference.

In [1601]:
clf = LogisticRegression(max_iter=1000000)
clf.fit(X_scaled[0:-100], Y_train[0:-100])
dev_pred_labels = clf.predict(X_test_scaled)

print('train_acc',clf.score(X_scaled[0:-100], Y_train[0:-100]))
print('test_acc',clf.score(X_test_scaled,Y_test))
print('left_out_set_acc',clf.score(X_scaled[-100:],Y_train[-100:]))

#print('acc', accuracy_score(Y_test ,dev_pred_labels))

#For partial dataset:
# train_acc 0.9802
# test_acc 0.4919735599622285

# for whole dataset:
#train_acc 0.7923755513547575
#test_acc 0.5873465533522191

#with standard scaling:
#train_acc 0.7864944339424491
#test_acc 0.6005665722379604

#and no shuffling
#train_acc 0.7827137156059651
#test_acc 0.6203966005665722

train_acc 0.7525476140276899
test_acc 0.6126804770872567
left_out_set_acc 0.62


Confusion matrix to see what is being misclassified below:

In [1585]:
import pandas
import seaborn
from sklearn.metrics import confusion_matrix
confusion_matrix(Y_test,dev_pred_labels)

array([[122,   2,   8,   4,   4,   7,  10,   0,   9,  12,   0],
       [ 10,  60,   7,   6,   1,   5,  16,   0,   0,   5,   0],
       [  8,  10, 117,   9,   6,  17,  15,   1,  10,  20,   0],
       [  7,   4,   6,  49,   7,   2,  12,   0,   6,  11,   0],
       [  6,   1,   1,   6,  41,   4,   8,   1,   2,   5,   0],
       [  7,   4,  12,   6,   0, 141,  25,   1,   4,  11,   0],
       [ 33,  10,  17,   4,   6,  19, 187,   0,   7,   8,   0],
       [  3,   0,   0,   2,   0,   2,   3,   8,   1,   1,   0],
       [  4,   3,   5,   2,   2,   5,  10,   2,  89,  12,   0],
       [ 15,   6,  12,  11,   9,   8,  18,   0,  14, 163,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   1]])

In [1586]:
print(np.shape(X_train))
print(np.shape(X_test))

print(np.shape(Y_traini))
print(np.shape(Y_testi))

(14329, 768)
(1593, 768)
(14329, 11)
(1593, 11)


Tried implementing a sequential layer with one dense layer (vocab size) and classification layer (with softmax)

In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional,Flatten, Dense, TimeDistributed, Dropout, Activation,SimpleRNN, LSTM

model = Sequential ()
#model.add(Bidirectional(LSTM(64, return_sequences = True)))
model.add(Dense(768, activation='linear'))
#model.add(Dropout(0.2))
#model.add(Bidirectional(LSTM(32, return_sequences = True)))
#model.add(Dense(11, activation='sigmoid'))
#model.add(Dropout(0.4))
#model.add(Bidirectional(LSTM(32, return_sequences = True)))
model.add(Dense(11, activation='softmax'))
#model.add(TimeDistributed(Dense(11, activation='softmax')))

In [22]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [23]:
model.build((14329, 768))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (14329, 768)              590592    
_________________________________________________________________
dense_1 (Dense)              (14329, 11)               8459      
Total params: 599,051
Trainable params: 599,051
Non-trainable params: 0
_________________________________________________________________


Output results below:

In [1597]:
history = model.fit(X_train, Y_traini, batch_size=32, epochs=400, validation_data=(X_test,Y_testi))

Epoch 1/400
448/448 [==============================] - 5s 8ms/step - loss: 3.9788 - accuracy: 0.5404 - val_loss: 1.5532 - val_accuracy: 0.5920
Epoch 2/400
448/448 [==============================] - 3s 7ms/step - loss: 1.3383 - accuracy: 0.6605 - val_loss: 1.2964 - val_accuracy: 0.6880
Epoch 3/400
448/448 [==============================] - 3s 6ms/step - loss: 1.1911 - accuracy: 0.6835 - val_loss: 1.3620 - val_accuracy: 0.6529
Epoch 4/400
448/448 [==============================] - 3s 6ms/step - loss: 1.1680 - accuracy: 0.6911 - val_loss: 1.3475 - val_accuracy: 0.6384
Epoch 5/400
448/448 [==============================] - 3s 7ms/step - loss: 1.1820 - accuracy: 0.6795 - val_loss: 1.4591 - val_accuracy: 0.6215
Epoch 6/400
448/448 [==============================] - 3s 6ms/step - loss: 1.1820 - accuracy: 0.6815 - val_loss: 1.4434 - val_accuracy: 0.6516
Epoch 7/400
448/448 [==============================] - 3s 7ms/step - loss: 1.1747 - accuracy: 0.6843 - val_loss: 1.4007 - val_accuracy: 0.6359

Epoch 58/400
448/448 [==============================] - 3s 6ms/step - loss: 1.0440 - accuracy: 0.6926 - val_loss: 1.6146 - val_accuracy: 0.6095
Epoch 59/400
448/448 [==============================] - 3s 6ms/step - loss: 1.0323 - accuracy: 0.7024 - val_loss: 1.5656 - val_accuracy: 0.6190
Epoch 60/400
448/448 [==============================] - 3s 6ms/step - loss: 1.0414 - accuracy: 0.6994 - val_loss: 1.5353 - val_accuracy: 0.6227
Epoch 61/400
448/448 [==============================] - 3s 6ms/step - loss: 1.0716 - accuracy: 0.6886 - val_loss: 1.6860 - val_accuracy: 0.6246
Epoch 62/400
448/448 [==============================] - 3s 6ms/step - loss: 1.0764 - accuracy: 0.6904 - val_loss: 1.5964 - val_accuracy: 0.6265
Epoch 63/400
448/448 [==============================] - 3s 6ms/step - loss: 1.0361 - accuracy: 0.6892 - val_loss: 1.6011 - val_accuracy: 0.5800
Epoch 64/400
448/448 [==============================] - 3s 6ms/step - loss: 1.0235 - accuracy: 0.6948 - val_loss: 1.5279 - val_accuracy:

Epoch 115/400
448/448 [==============================] - 3s 6ms/step - loss: 1.0020 - accuracy: 0.7047 - val_loss: 1.6841 - val_accuracy: 0.5788
Epoch 116/400
448/448 [==============================] - 3s 6ms/step - loss: 1.0495 - accuracy: 0.6882 - val_loss: 1.7650 - val_accuracy: 0.6077
Epoch 117/400
448/448 [==============================] - 3s 6ms/step - loss: 1.0089 - accuracy: 0.7058 - val_loss: 1.6619 - val_accuracy: 0.5976
Epoch 118/400
448/448 [==============================] - 3s 6ms/step - loss: 0.9868 - accuracy: 0.7087 - val_loss: 1.6225 - val_accuracy: 0.6077
Epoch 119/400
448/448 [==============================] - 3s 6ms/step - loss: 1.0100 - accuracy: 0.7033 - val_loss: 1.7078 - val_accuracy: 0.6089
Epoch 120/400
448/448 [==============================] - 3s 6ms/step - loss: 1.0239 - accuracy: 0.6952 - val_loss: 1.7113 - val_accuracy: 0.6095
Epoch 121/400
448/448 [==============================] - 3s 6ms/step - loss: 1.0351 - accuracy: 0.6954 - val_loss: 1.7348 - val_ac

448/448 [==============================] - 3s 8ms/step - loss: 0.9805 - accuracy: 0.7044 - val_loss: 1.8383 - val_accuracy: 0.5945
Epoch 228/400
448/448 [==============================] - 3s 6ms/step - loss: 0.9977 - accuracy: 0.7019 - val_loss: 1.8900 - val_accuracy: 0.5687
Epoch 229/400
448/448 [==============================] - 3s 7ms/step - loss: 0.9952 - accuracy: 0.7029 - val_loss: 1.7922 - val_accuracy: 0.5851
Epoch 230/400
448/448 [==============================] - 5s 10ms/step - loss: 0.9971 - accuracy: 0.7033 - val_loss: 1.7687 - val_accuracy: 0.5995
Epoch 231/400
448/448 [==============================] - 4s 9ms/step - loss: 0.9843 - accuracy: 0.7082 - val_loss: 1.8323 - val_accuracy: 0.5650
Epoch 232/400
448/448 [==============================] - 3s 6ms/step - loss: 0.9904 - accuracy: 0.7115 - val_loss: 1.8873 - val_accuracy: 0.5794
Epoch 233/400
448/448 [==============================] - 3s 7ms/step - loss: 1.0178 - accuracy: 0.6996 - val_loss: 1.7985 - val_accuracy: 0.604

448/448 [==============================] - 3s 7ms/step - loss: 0.9906 - accuracy: 0.7030 - val_loss: 1.9222 - val_accuracy: 0.5656
Epoch 340/400
448/448 [==============================] - 3s 7ms/step - loss: 1.0002 - accuracy: 0.7018 - val_loss: 1.8481 - val_accuracy: 0.6240
Epoch 341/400
448/448 [==============================] - 4s 9ms/step - loss: 1.0033 - accuracy: 0.7054 - val_loss: 1.8810 - val_accuracy: 0.5744
Epoch 342/400
448/448 [==============================] - 3s 7ms/step - loss: 0.9931 - accuracy: 0.7036 - val_loss: 1.8835 - val_accuracy: 0.5964
Epoch 343/400
448/448 [==============================] - 4s 8ms/step - loss: 1.0065 - accuracy: 0.6990 - val_loss: 1.9973 - val_accuracy: 0.6014
Epoch 344/400
448/448 [==============================] - 3s 7ms/step - loss: 1.0046 - accuracy: 0.7067 - val_loss: 1.8733 - val_accuracy: 0.6190
Epoch 345/400
448/448 [==============================] - 3s 8ms/step - loss: 0.9871 - accuracy: 0.7102 - val_loss: 1.8862 - val_accuracy: 0.5857

Trial to predict a section taken out from of X_train to see if results are the same

In [1456]:
# Xnew= X_train[-500:]
# ynew = np.argmax(model.predict(Xnew), axis=-1)

# for i in range(len(Xnew)):
# 	print("Predicted=%s, Actual=%s" % ( ynew[i],Y_traini[-500:][i]))
 
# #print(ynew)